In [2]:
import pandas as pd
import numpy as np
import scipy as scp
import math

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy import stats

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler

In [13]:
# data = pd.read_csv('RUvideos.csv')
# data = pd.read_csv('musicnet.csv')
data = pd.read_csv('covid19_tweets.csv')
print(data.info())
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179108 entries, 0 to 179107
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         179108 non-null  object
 1   user_location     142337 non-null  object
 2   user_description  168822 non-null  object
 3   user_created      179108 non-null  object
 4   user_followers    179108 non-null  int64 
 5   user_friends      179108 non-null  int64 
 6   user_favourites   179108 non-null  int64 
 7   user_verified     179108 non-null  bool  
 8   date              179108 non-null  object
 9   text              179108 non-null  object
 10  hashtags          127774 non-null  object
 11  source            179031 non-null  object
 12  is_retweet        179108 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 9.9+ MB
None


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,['WearAMask'],Twitter for Android,False
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,['COVID19'],Twitter for Android,False
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,NaN,Twitter Web App,False
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",['COVID19'],Twitter for iPhone,False


In [34]:
print(data['user_location'].value_counts()[:20])

sample_india = data[data['user_location'] == 'India'].sample(500, random_state=13)
sample_us = data[(data['user_location'] == 'United States') | (data['user_location'] == 'US')].sample(500, random_state=13)
sample_uk = data[(data['user_location'] == 'United Kingdom') | (data['user_location'] == 'UK')].sample(500, random_state=13)
sample_canada = data[data['user_location'] == 'Canada'].sample(500, random_state=13)
sample_sa = data[data['user_location'] == 'South Africa'].sample(500, random_state=13)

print('\n' + str(len(sample_india)), len(sample_us), len(sample_uk), len(sample_canada), len(sample_sa))

India               3741
United States       2455
New Delhi, India    1721
Mumbai, India       1401
Washington, DC      1354
London, England     1269
United Kingdom      1206
London              1103
New York, NY        1082
Australia            964
Worldwide            879
Canada               835
New Delhi            822
USA                  817
UK                   803
WORLDWIDE            777
Los Angeles, CA      776
South Africa         766
Lagos, Nigeria       731
California, USA      709
Name: user_location, dtype: int64

500 500 500 500 500


In [20]:
# Задание 1

normality = {}
alpha = 0.05

for column_title in data.columns:
    print(column_title)
    normality[column_title] = stats.shapiro(data[column_title])
#     Тест Шапиро-Уилка на нормальность распределения
    print(stats.shapiro(data[column_title]))
    if normality[column_title].pvalue >= alpha:
        print('Нормальное распределение\n')
    else:
        print('Распределение не является нормальным\n')

user_name


ValueError: could not convert string to float: ' 2Civility'